In [2]:
from pint import UnitRegistry
import pandas as pd


# helpef functions, constants, etc 

In [9]:
class ComponentReportClass:
    def __init__(self, depth, length, width):
        self.depth = depth
        self.length = length
        self.width = width 
        self.rebar = None
    # dimensional calcs   
    def calc_volume(self):
        return (self.depth).to("yards") * (self.width).to("yards") * (self.length).to("yards")
    def calc_top_face_area(self):
        return self.length * self.width 
    def calc_sides_face_area(self):
        return 2 * (self.depth * self.width ) + 2 * (self.depth * self.length )
    # conversion calcs
    def to_cubic_yards(a):
        return a/27

    # basic work calcs
    def calc_subgrade_prep(self):
        bottom = self.calc_top_face_area()
        sides = self.calc_sides_face_area()
        return bottom + sides
    def calc_formwork(self):
        return self.calc_sides_face_area()
    def calc_excavation(self):
        # TODO add benching...
        return self.calc_volume()
    def calc_concrete(self):
        return self.calc_volume()* 1.08


    # rebar calcs
    def calc_rebar(self, quant, length):
        """quant and length need to be arrays """
        quant = to_list(quant)
        length = to_list(length)

        h = [a*b for a,b in zip(quant, length)]
        self.rebar = sum(h)
        return self.rebar

    # reports
    def report_basic(self):
        report = {
            "subgrade": self.calc_subgrade_prep(),
            "excavation": self.calc_excavation(),
            "formwork": self.calc_formwork(),
            "rebar_template": 0*ft,
            "concrete": self.calc_concrete()
        }
        if self.rebar:
            report["rebar"] = self.rebar
        return report 


    

In [4]:
u = UnitRegistry()
ft = u.feet
inc = u.inches

def to_list(a):
    a = [a] if type(a) != list else a
    return a

def calc_component_totals(components, fx):
    """components are defined components
    fx is a function that takes in a list of components """
    c_reports = [c.report_basic() for c in components]

    sum_components = {}
    for k in c_reports[0].keys():
        sum_components[k] = fx(c_reports, k)

    return sum_components

def to_df(a_dict):
    t1 = { f"{k} [{str(v.units)}]": round(v.magnitude,2) for k,v in a_dict.items()}
    return pd.DataFrame(t1, index=[0])

rebar types

In [5]:
rebar_dict = {
    "num_5": 4*ft + 9*inc
}

# Start Calcs

In [10]:
# pile caps 
pc1 = ComponentReportClass(2*ft, 4*ft, 4*ft) # reported in ft!

pc2 = ComponentReportClass(2*ft + 9*inc, 
                            17*ft + 2*inc, 
                            10*ft + 6*inc) 
pc1.calc_rebar(14, rebar_dict["num_5"]) 
pc2.calc_rebar(14+16, rebar_dict["num_5"])

pc = calc_component_totals([pc1, pc2], lambda list, key: 2*list[0][key] + list[1][key] )

to_df(pc)

,subgrade [foot ** 2],excavation [yard ** 3],formwork [foot ** 2],rebar_template [foot],concrete [yard ** 3],rebar [foot]
0,428.42,20.73,216.17,0,22.39,275.5


In [11]:
# gradebeams
gb_length = 6 * (2*ft + (6*inc + (1/8)*inc))
gb_width = gb_height = (12*inc).to("feet")
gb1 = ComponentReportClass(gb_width, gb_length,  gb_height) 
gb1.calc_rebar([6,80], [rebar_dict["num_5"], 1*ft])
gb = calc_component_totals([gb1], lambda list, key: 2*list[0][key])
to_df(gb)

,subgrade [foot ** 2],excavation [yard ** 3],formwork [foot ** 2],rebar_template [foot],concrete [yard ** 3],rebar [foot]
0,94.38,1.12,64.25,0,1.21,217.0


# calc all component totals

In [12]:
# all 
def fx(list, key):
    # 2* pc1 + 1*pc2 * 2*gb1
   return 2*list[0][key] + list[1][key] + 2*list[2][key]

t = calc_component_totals([pc1, pc2, gb1], fx) # TODO update excel!!!
to_df(t)

,subgrade [foot ** 2],excavation [yard ** 3],formwork [foot ** 2],rebar_template [foot],concrete [yard ** 3],rebar [foot]
0,522.79,21.84,280.42,0,23.59,492.5
